## AIR Rerun EP (working).ipynb

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# AIR Rerun EP

# In[14]:


import pandas as pd
import numpy as np
import os
import pyodbc


# In[15]:


#create EP table
cols = ['ID','ANLSID','NAME','EPTYPE','PERSPCODE','RP','PERSPVALUE']
ep_table = pd.DataFrame(columns=cols).set_index('ID')
ep_table.head()


# In[16]:


server = 'DFWCAT-TS3SQL1'
db = 'RV_CM_locAAL_TSR'
cnxn = pyodbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+db+';')


# In[17]:


#Determine Loss to use
AnalysisSID_query = "select * from tAnalysisResult"
AnalysisSID = pd.read_sql(AnalysisSID_query,cnxn)
AnalysisSID.head()


# In[18]:


AnalysisSID = 6


# In[19]:


ResultSplitLevel = 'State'


# In[20]:


#Determine loss perspectives saved
perspectives_query = "SELECT distinct FinancialPerspectiveCode from t6_LOSS_AnnualEP" #update loss table number
perspectives = pd.read_sql(perspectives_query,cnxn)

perspectives = pd.DataFrame(perspectives)
perspectives.head()


# In[21]:


#Get details of the analysis
AnalysisName_query = "SELECT AnalysisName FROM tAnalysisResult WHERE ResultSID=6" #update loss table number
AnalysisName = pd.read_sql(AnalysisName_query,cnxn).iloc[0,0]
OuputTypeCode_query = 'SELECT OutputTypeCode FROM tLossAnalysisOption WHERE ResultSID=6' #update loss table number
OutputTypeCode = pd.read_sql(OuputTypeCode_query,cnxn).iloc[0,0]
GeoLevelCode_query = 'SELECT GeoLevelCode FROM tLossAnalysisOption WHERE ResultSID=6'
GeoLevelCode = pd.read_sql(GeoLevelCode_query,cnxn).iloc[0,0]
print(AnalysisName)
print(OutputTypeCode)
print(GeoLevelCode)


# In[22]:


#Set AnalysisLevel
if OutputTypeCode == 'LOC':
    AnalysisLevel = 1
elif OutputTypeCode == 'GEOL' and GeoLevelCode == 'SUBA':
    AnalysisLevel = 2
elif OutputTypeCode == 'GEOL' and GeoLevelCode == 'AREA':
    AnalysisLevel = 3
elif OuputTypeCode == 'CONGEAL' and GeoLevelCode == 'SUBA':
    AnalysisLevel = 4
elif OutputTypeCode == 'EAGEOL' and GeoLevelCode == 'SUBA':
    AnalysisLevel = 5
elif OuputTypeCode == 'CONGEAL' and GeoLevelCode == 'AREA': 
    AnalysisLevel = 6
elif OutputTypeCode == 'EAGEOL' and GeoLevelCode == 'AREA':
    AnalysisLevel = 7
elif OutputTypeCode == 'EA':
    AnalysisLevel = 8
elif OutputTypeCode == 'CON':
    AnalysisLevel = 9
else:
    AnalysisLevel = None


# In[23]:


#Set ResultTable

if AnalysisLevel == 1:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByLocation'
elif AnalysisLevel == 2:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByGeo'
elif AnalysisLevel == 4:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByContractGeo'
elif AnalysisLevel == 5:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByExposureAttributeGeo'
elif AnalysisLevel == 8:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByByExposureAttribute'
elif AnalysisLevel == 9:
    ResultTable = 't'+str(AnalysisSID)+'_LOSS_ByContract'
else:
    ResultTable = None


# In[24]:


#Set current max SID
CurrentMaxSID_query = "SELECT LastSID FROM tSIDControl WHERE TableName='tAnalysisResult'"
CurrentMaxSID = pd.read_sql(CurrentMaxSID_query,cnxn).iloc[0,0]
print(CurrentMaxSID)


# In[25]:


ResultTable


# In[26]:


##Now we do the by State split
#Drop temp_geo table if it exists
# drop_temp_geo_query = 'drop table temp_geo'
# drop_temp_geo = pd.read_sql(drop_temp_geo_query,cnxn)

#Insert all states into temp_geo
if AnalysisLevel == 1:
    ResultTable = 't6_LOSS_ByLocation'+' z INNER JOIN t'+str(AnalysisSID)+'_LOSS_DimLocation a ON z.LocationSID=a.LocationSID'
else:
    ResultTable = 't6_LOSS_ByLocation' + ' a'


# In[ ]:


#couldn't get this to work
# temp_geo_query = '''SELECT AreaCode State,AreaCode Geo into temp_geo FROM t6_LOSS_ByLocation z INNER JOIN t6_LOSS_DimLocation a ON z.LocationSID=a.LocationSID INNER JOIN AIRGeography.dbo.tGeography g ON a.GeographySID=g.GeographySID GROUP BY AreaCode'''
# temp_geo = pd.read_sql(temp_geo_query,cnxn)
# temp_geo = pd.DataFrame(temp_geo)


# In[ ]:


#temp_geo.to_sql(temp_geo, con=cnxn)
#this killed kernal so created table in SSMS


# In[32]:


temp_geo_query = 'select * from temp_geo'
temp_geo = pd.read_sql(temp_geo_query,cnxn)
print(temp_geo)


# In[ ]:


#insert all counties into temp_geo
# temp_geo_query = '''SELECT AreaCode State,SubAreaName Cpunty, AreaCode+"_"+SubAreaName Geo into temp_geo FROM t6_LOSS_ByLocation z INNER JOIN t6_LOSS_DimLocation a ON z.LocationSID=a.LocationSID INNER JOIN AIRGeography.dbo.tGeography g ON a.GeographySID=g.GeographySID GROUP BY AreaCode'''
# temp_geo = pd.read_sql(temp_geo_query,cnxn)
# temp_geo = pd.DataFrame(temp_geo)


# In[37]:


##Create Master Loss Table
#Loss By Event

#Couldn't get this to work either
# temp_eventloss_query = '''SELECT CatalogTypeCode,EventID,ModelCode,YearID,PerilSetCode,Geo, 
# sum(GroundUpLoss) GroundUpLoss,sqrt(sum(GroundUpSD*GroundUpSD)) GroundUpSD,sum(GroundUpMaxLoss) GroundUpMaxLoss,
# sum(GrossLoss) GrossLoss,sqrt(sum(GrossSD*GrossSD)) GrossSD,sum(GrossMaxLoss) GrossMaxLoss,
# sum(NetOfPreCatLoss) NetOfPreCATLoss
# INTO temp_eventloss
# FROM t6_LOSS_ByLocation z INNER JOIN t6_LOSS_DimLocation a ON z.LocationSID=a.LocationSID INNER JOIN AirGeography.dbo.tGeography g ON a.GeographySID=g.GeographySID
# INNER JOIN temp_geo tg ON g.AreaCode=tg.State
# GROUP BY CatalogTypeCode,EventID,ModelCode,YearID,PerilSetCode,Geo'''
# temp_eventloss = pd.read_sql(temp_eventloss_query,cnxn)
# print(temp_eventloss)


# In[ ]:


print(temp_eventloss)



## Bulk_Insert_SQL.py

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

server = 'GCVPW-RDB-HAPRZ' #edit server name
db = 'AccumTesting_v21_EDM' #edit db name

params = urllib.parse.quote_plus('DRIVER={SQL SERVER};SERVER= '+server+';DATABASE='+db+';')

df = pd.read_csv('C:\Python\Colonnade_Galahad_Exposure.txt', sep = '\t')

engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params), fast_executemany=True)

df.to_sql('Colonnade_Galahad_Exposure', con = engine, index=False, if_exists='append',schema='dbo')

## Convert_CSV_to_TXT.ipynb

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[13]:


import os
import csv
from pathlib import Path


# In[4]:


directory = r'C:\Users\u1207199\Downloads\Arium\Everest\ELDs'


# In[15]:


for in_path in Path(directory).glob('*.csv'):
    out_path = in_path.with_suffix('.txt')
    with in_path.open('r') as fin, out_path.open('w') as fout:
        reader = csv.DictReader(fin)
        writer = csv.DictWriter(fout, reader.fieldnames, delimiter='\t')
        writer.writeheader()
        writer.writerows(reader)


# In[ ]:






## Export_SQL_Table.py

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

server = 'GCVPW-TDB-DZ5GK' #add server name here
DB= 'AIRResult_Secura_06302023_TSv10' #add database name here

engine = create_engine(f'mssql+pyodbc://{server}/{DB}?driver=SQL+Server')

sql = '''select * from 
				SECURA_SCS_YLT	--add table name here
			'''

df = pd.read_sql_query(sql, engine)

df.to_csv(r'C:\Python\SECURA_SCS_YLT.csv', index=False)	#add file name here


# import pandas as pd
# import pyodbc

# server = 'GCVPW-TDB-P5BQQ' #add server name here
# DB= 'Hagerty_20221231_v10_TSE' #add database name here


# conn = pyodbc.connect('DRIVER={SQL Server}; SERVER='+server+';DATABASE='+DB+';')

# sql = pd.read_sql_query('''select * from 
# 				z_GCAP_Upload	--add table name here
# 			''', conn)

# df = pd.DataFrame(sql)
# df.to_csv(r'C:\Python\Hagerty_20221231_GCAP_ExposureOnly.csv', index=False)	#add file name here

## Import Data to SQL Database.py

In [ ]:
#Import Excel or CSV file to SQL Server database

import pandas as pd
import pypyodbc as odbc
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import url

server = '' #add server name here
database = '' #add database here
driver = '{SQL Server}'

connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': f'DRIVER{driver};SERVER={server};DATABASE={database}'})
engine = create_engine(connection_url, module=odbc)

file = '' #add excel or csv file here
df = pd.read_excel(file, sheet_name=None)
for sheet_name, table in df.items():
    if table.empty:
        continue
    try:
        df.to_sql(sheet_name, con=engine, if_exists='replace', index=False, dtype={'user_id': sqlalchemy.types.VARCHAR(length=255)})

## Import Data to SQL.py

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

server_name = 'DFWCAT-TS5SQL2'
database_name = 'UticaNational_20220930_v9_TSE_AIRExp'

# set up the connection to the SQL Server database
engine = create_engine(f'mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')

# read the .csv file into a pandas DataFrame
df = pd.read_csv('AIR_ConstScheme_ForSQL.csv')

# write the data from the DataFrame to the SQL Server table
df.to_sql('table_name', engine, if_exists='append', index=False)


## pandas - applying bands.py

In [ ]:
def yr_blt(data):
    if data.Year_Built <= 1975:
        return 'Pre-1975'
    elif data.Year_Built >1975 and data.Year_Built <= 1994:
        return '1976-1994'
    elif data.Year_Built > 1994 and data.Year_Built <= 1997:
        return '1995-1997'
    elif data.Year_Built > 1997 and data.Year_Built <= 2001:
        return '1998-2001'
    elif data.Year_Built > 2001 and data.Year_Built <=2008:
        return '2002-2008'
    elif data.Year_Built > 2008:
        return 'Post-2009'
    else:
        return 'Unknown'
df['Year_Band'] = df.apply(yr_blt, axis='columns')

## Print Full Pandas DF.py

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

#OR

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


RMS Layer Stdev

In [ ]:
from scipy.stats import betainc
from math import sqrt

def RMSLayerStdev(AttPt, limit, mean, stddev, Exposure):
    if Exposure == 0 or stddev == 0:
        return 0

    mu = mean / Exposure
    sigma = stddev / Exposure

    Alpha = (mu ** 2 * (1 - mu) / sigma ** 2) - mu
    Beta = Alpha * (1 - mu) / mu

    L2 = min(1, AttPt / Exposure)
    L3 = min(1, (AttPt + limit) / Exposure)

    r2 = mu * (1 - betainc(Alpha + 1, Beta, L2)) - L2 * (1 - betainc(Alpha, Beta, L2))
    r3 = mu * (1 - betainc(Alpha + 1, Beta, L3)) - L3 * (1 - betainc(Alpha, Beta, L3))

    xsq2 = (Alpha + 1) * mu * (1 - betainc(Alpha + 2, Beta, L2)) / (Alpha + Beta + 1) - 2 * L2 * mu * (1 - betainc(Alpha + 1, Beta, L2)) + L2 ** 2 * (1 - betainc(Alpha, Beta, L2))
    xsq3 = (Alpha + 1) * mu * (1 - betainc(Alpha + 2, Beta, L3)) / (Alpha + Beta + 1) - 2 * L3 * mu * (1 - betainc(Alpha + 1, Beta, L3)) + L3 ** 2 * (1 - betainc(Alpha, Beta, L3))

    lxsq = xsq2 - xsq3 - 2 * (L3 - L2) * r3

    return sqrt(max(0, (lxsq - (r2 - r3) ** 2))) * Exposure
